Create a Neural Network from Scratch


Create a layer class

In [310]:
import numpy as np

# Define a base class for layers of the neural network
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward(self, input):
        # This method will be overridden by subclasses
        pass

    def backward(self, grad_output, lr):
        # This method will be overridden by subclasses
        pass


## Create a Dense Layer


In [311]:
# Define a class for Dense layers
class Dense(Layer):
    def __init__(self, input_size, output_size):
        # Initialize weights and biases
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.random.randn(output_size, 1)
    
    def forward(self, input):
        # Store input and compute output
        self.input = input
        self.output = np.dot(input, self.weights.T) + self.biases.T
        return self.output

    def backward(self, grad_output, lr):
        # Compute gradient with respect to input
        grad_input = np.dot(grad_output, self.weights)

        # Compute gradient with respect to weights and biases
        grad_weights = np.dot(self.input.T, grad_output)
        grad_biases = np.sum(grad_output, axis=0, keepdims=True)

        # Update weights and biases
        self.weights -= lr * grad_weights.T
        self.biases -= lr * grad_biases.T

        return grad_input


Create an Activation Layer

In [312]:
# Define a base class for activation functions
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        # Store input and compute output
        self.input = input
        return self.activation(self.input)
    
    def backward(self, output_gradient, learning_rate):
        # Compute gradient with respect to input
        return output_gradient * self.activation_prime(self.input)

## Create Instances of an Activation layer
I used Tanh and Sigmoid


In [313]:
# Define a class for Tanh activation function
class Tanh(Activation):
    def __init__(self):
        tanh = lambda x: np.tanh(x)
        tanh_prime = lambda x: 1 - np.tanh(x) ** 2
        super().__init__(tanh, tanh_prime)

# Define a function for computing Mean Squared Error
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

# Define a function for computing the derivative of Mean Squared Error
def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

In [314]:
class Sigmoid(Activation):
    def __init__(self):
        pass

    def forward(self, inputs):
        return 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues, inputs):
        sigmoid = self.forward(inputs)
        return dvalues * sigmoid * (1 - sigmoid)


## Create a NeuralNetwork


In [315]:
# Define a class for the neural network
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers

    def forward(self, input_data):
        # Pass the input data through all layers
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

    def backward(self, output_gradient, learning_rate):
        # Pass the output gradient through all layers in reverse order
        for layer in reversed(self.layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
        return output_gradient

    def train(self, x_train, y_train, epochs, learning_rate):
        # Train the network for a specified number of epochs
        for i in range(epochs):
            output = self.forward(x_train)
            error = mse(y_train, output)
            print('Epoch %d/%d, Error: %f' % (i + 1, epochs, error))
            error_gradient = mse_prime(y_train, output)
            self.backward(error_gradient, learning_rate)



## Create some tests

In [316]:
# Test functions
def test_layers():
    # Test the Dense layer, Tanh and Sigmoid activation functions
    dense = Dense(4, 4)
    tanh = Tanh()
    sigmoid = Sigmoid()
    input_data = np.array([1, 2, 3, 4])
    output = dense.forward(input_data)
    print("Dense layer output: ", output)
    output = tanh.forward(output)
    print("Tanh activation output: ", output)
    output = sigmoid.forward(output)
    print("Sigmoid activation output: ", output)

# Test Neural Network
def test_neural_network():
    # Define the layers of the network
    layers = [Dense(4, 3), Tanh(), Dense(3, 1), Sigmoid()]
    # Create the network
    nn = NeuralNetwork(layers)
    # Define training data
    x_train = np.array([[1, 2, 3, 4], [2, 3, 4, 5]])
    y_train = np.array([[0.5], [0.6]])
    # Train the network
    nn.train(x_train, y_train, epochs=10, learning_rate=0.05)

# Run the test functions
test_layers()
test_neural_network()


Dense layer output:  [[ 4.31930414 -7.20339626 -2.34658165 -8.37067502]]
Tanh activation output:  [[ 0.9996458  -0.99999889 -0.98185086 -0.99999989]]
Sigmoid activation output:  [[0.73098893 0.26894164 0.27252469 0.26894144]]
Epoch 1/10, Error: 0.071552
Epoch 2/10, Error: 0.062066
Epoch 3/10, Error: 0.051604
Epoch 4/10, Error: 0.040827
Epoch 5/10, Error: 0.030768
Epoch 6/10, Error: 0.022429
Epoch 7/10, Error: 0.016242
Epoch 8/10, Error: 0.011971
Epoch 9/10, Error: 0.009100
Epoch 10/10, Error: 0.007158
